In [1]:
# Want to make sure the Zarred point clouds I'm putting in the dataset match the distribution of
# expected points in Cheng's dataset

In [2]:
import zarr
import numpy as np

In [3]:
z = zarr.open("/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/data/garmentnets_tracking_simulation_dataset_shortened.zarr")
z_orig = zarr.open("/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/data/garmentnets_simulation_dataset_sample.zarr")

In [4]:
for sample_key in ['00380_Tshirt_509', '00471_Tshirt_153']:
    print("Sample:", sample_key)
    print(80 * '-')
    grasped_index = int(sample_key.split('_')[-1])
    cloud_cheng = z_orig[f"/Tshirt/samples/{sample_key}/point_cloud/point"][:]
    rgb_cheng = z_orig[f"/Tshirt/samples/{sample_key}/point_cloud/rgb"][:]
    cloud_mine = z[f"/Tshirt/samples/{sample_key}/point_cloud/point"][:]
    rgb_mine = z[f"/Tshirt/samples/{sample_key}/point_cloud/rgb"][:]

    print('XYZ Comparison at t=0')
    print(f"\tCheng's cloud stats: | min: {cloud_cheng.min(axis=0)} | max: {cloud_cheng.max(axis=0)} | std: {cloud_cheng.std(axis=0)}")
    print(f"\tOur cloud stats: | min: {cloud_mine.min(axis=0)} | max: {cloud_mine.max(axis=0)} | std: {cloud_mine.std(axis=0)}")

    print('RGB Comparison at t=0')
    print(f"\tCheng's cloud stats: | min: {rgb_cheng.min(axis=0)} | max: {rgb_cheng.max(axis=0)} | std: {rgb_cheng.std(axis=0)}")
    print(f"\tOur cloud stats: | min: {rgb_mine.min(axis=0)} | max: {rgb_mine.max(axis=0)} | std: {rgb_mine.std(axis=0)}")

    dynamics_key = f"/Tshirt/samples/{sample_key}/dynamics/0/point_cloud/timestep_50"
    dynamics_clouds = []
    rgb_clouds = []
    for i in range(4):
        dynamics_cloud_group = z[dynamics_key + f"/view_{i}/point"]
        # print(dynamics_cloud_group)
        dynamics_cloud = dynamics_cloud_group[:]
        dynamics_clouds.append(dynamics_cloud)
        
        rgb_cloud = z[dynamics_key + f"/view_{i}/rgb"][:]
        rgb_clouds.append(rgb_cloud)
        


        # print(f"\tOur t=50 view_{i} cloud stats: | min: {dynamics_cloud.min()} | max:
        # {dynamics_cloud.max()} | std: {dynamics_cloud.std()}")
    rgb_clouds = np.concatenate(rgb_clouds, axis=0)
    dynamics_clouds = np.concatenate(dynamics_clouds, axis=0)
    print(f"Our t=50 XYZ cloud stats: | min: {dynamics_clouds.min(axis=0)} | max: {dynamics_clouds.max(axis=0)} | std: {dynamics_clouds.std(axis=0)}")
    print(f"Our t=50 RGB cloud stats: | min: {rgb_clouds.min(axis=0)} | max: {rgb_clouds.max(axis=0)} | std: {rgb_clouds.std(axis=0)}")

    gripper_deltas = z[f"/Tshirt/samples/{sample_key}/dynamics/0/delta_gripper_pos"][:]
    
    gripper_translations = np.cumsum(gripper_deltas, axis=0) + np.array((0,0,0.4))
    print("Translating dynamics configuration back to origin")
    print("\tUsing gripper deltas:")
    trans = gripper_translations[50, :]
    dynamics_clouds_translated = dynamics_clouds - trans
    print(f"\tmin: {dynamics_clouds_translated.min(axis=0)} | max: {dynamics_clouds_translated.max(axis=0)} | std: {dynamics_clouds_translated.std(axis=0)}")

    print("\tUsing grasped index")
    trans = dynamics_clouds[grasped_index, :] - cloud_mine[grasped_index, :]
    dynamics_clouds_translated = dynamics_clouds - trans
    print(f"\tmin: {dynamics_clouds_translated.min(axis=0)} | max: {dynamics_clouds_translated.max(axis=0)} | std: {dynamics_clouds_translated.std(axis=0)}")
    # print(f"Our t=50 view_{i} translated cloud stats: | min: {dynamics_clouds_translated.min(axis=0)} | max: {dynamics_clouds_translated.max(axis=0)} | std: {dynamics_clouds_translated.std(axis=0)}")

    print()

Sample: 00380_Tshirt_509
--------------------------------------------------------------------------------
XYZ Comparison at t=0
	Cheng's cloud stats: | min: [-0.1843 -0.1121 -0.6685] | max: [ 0.1622    0.1101   -0.005077] | std: [0.02806 0.01872 0.1059 ]
	Our cloud stats: | min: [-0.11523 -0.128   -0.673  ] | max: [ 0.1665   0.1382  -0.00547] | std: [0.02615 0.02615 0.1046 ]
RGB Comparison at t=0
	Cheng's cloud stats: | min: [ 3 13  7] | max: [143 186 159] | std: [17.36414687 20.83240303 19.66328287]
	Our cloud stats: | min: [ 8 17 17] | max: [133 186 151] | std: [15.0085898  18.58095829 17.494503  ]
Our t=50 XYZ cloud stats: | min: [-0.1094 -0.0762 -0.2013] | max: [0.1467 0.1296 0.4768] | std: [0.02692 0.01906 0.0761 ]
Our t=50 RGB cloud stats: | min: [ 5 22 12] | max: [135 186 153] | std: [14.72235304 18.37958582 17.25090597]
Translating dynamics configuration back to origin
	Using gripper deltas:
	min: [-0.1331468  -0.16258747 -0.687464  ] | max: [ 0.12295671  0.04322308 -0.00936341

In [36]:
import open3d as o3d

# xyz_data = np.concatenate((cloud_cheng, dynamics_clouds), axis=0)
xyz_data = o3d.utility.Vector3dVector(dynamics_clouds)
rgb_data = o3d.utility.Vector3dVector(rgb_clouds / 255.)

pcd = o3d.geometry.PointCloud()
pcd.colors = o3d.utility.Vector3dVector(rgb_data)
pcd.points = o3d.utility.Vector3dVector(xyz_data)
o3d.visualization.draw_geometries([pcd])

## Reading A Dynamics Point Cloud From Zarr And Visualizing Via Open3D

In [3]:
z = zarr.open("/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/data/garmentnets_tracking_simulation_dataset_shortened.zarr")

# Get the point clouds for each of the 4 views.
dynamics_key = f"/Tshirt/samples/00471_Tshirt_153/dynamics/0/point_cloud/timestep_50"
dynamics_clouds = []
rgb_clouds = []
for i in range(4):
    dynamics_cloud_group = z[dynamics_key + f"/view_{i}/point"]
    # print(dynamics_cloud_group)
    dynamics_cloud = dynamics_cloud_group[:]
    dynamics_clouds.append(dynamics_cloud)
    
    rgb_cloud = z[dynamics_key + f"/view_{i}/rgb"][:]
    rgb_clouds.append(rgb_cloud)

# Concatenate the point clouds to get one full pointcloud
rgb_clouds = np.concatenate(rgb_clouds, axis=0)
dynamics_clouds = np.concatenate(dynamics_clouds, axis=0)

import open3d as o3d

xyz_data = o3d.utility.Vector3dVector(dynamics_clouds)
rgb_data = o3d.utility.Vector3dVector(rgb_clouds / 255.)

pcd = o3d.geometry.PointCloud()
pcd.colors = o3d.utility.Vector3dVector(rgb_data)
pcd.points = o3d.utility.Vector3dVector(xyz_data)
o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
